Here we calculate the hardness parameter (HP) of a given problem instace as defined by Ebadi et. al (https://arxiv.org/pdf/2202.09372.pdf) where HP = $\frac{D_{|MIS| - 1}}{(|MIS| \cdot D_{|MIS|})}$ 

In [1]:
using Bloqade
using Random
Random.seed!(42)
using PythonCall
plt = pyimport("matplotlib.pyplot");
np = pyimport("numpy")

# using Graphs
using GenericTensorNetworks, Graphs
using GenericTensorNetworks: unit_disk_graph
using Optim


SyntaxError: invalid syntax (3340530492.py, line 1)

In [ ]:
# Set the random seed for reproducibility
Random.seed!(2)

# Base scale factor
a = 1

# Rb=(40)^(0.25) * a

# Set blockade radius to 3 times the scale factor 'a'
Rb = 3 * a

# Set the dimension for the lattice
dimension = 5

# Initialize arrays to store results using numpy (np should be previously aliased to PyCall's numpy)
Rb_arr = np.array([])
dimension_arr = np.array([])
HP_arr = np.array([])

function plot(Rb, dimension, Rb_arr, dimension_arr, HP_arr)
    println(Rb, " ", dimension)
    
    # Generate sites on a square lattice, apply random dropout of 20%
    atoms = generate_sites(SquareLattice(), dimension, dimension; scale = a) |> random_dropout(0.2)

    # Bloqade.plot(atoms, blockade_radius = Rb)

    # Generate a graph based on the unit disk graph model
    graph = BloqadeMIS.unit_disk_graph(atoms, Rb)

    # Define an IndependentSet problem with the TreeSA optimizer
    problem = IndependentSet(graph; optimizer=TreeSA())

    # maximum_independent_set_size = GenericTensorNetworks.solve(problem, SizeMax())[]

    # Solve the MIS problem to count the maximum independent sets
    count_maximum_independents_sets = GenericTensorNetworks.solve(problem, CountingMax())[]

    # Solve to count maximum independent sets with a size constraint
    count_max2_independent_sets = GenericTensorNetworks.solve(problem, CountingMax(2))[]

    # Extract the number of MIS and its degeneracy
    MIS = count_maximum_independents_sets.n
    DMIS = count_maximum_independents_sets.c
    DMIS_minus1 = count_max2_independent_sets.coeffs[1]

    # Calculate Hardness Parameter
    HP = DMIS_minus1 / (MIS * DMIS)

    # Print the HP value
    println(HP)

    # Append the current results to the result arrays
    global Rb_arr = np.append(Rb_arr, Rb)
    global dimension_arr = np.append(dimension_arr, dimension)
    global HP_arr = np.append(HP_arr, HP)
end

# Determine which parameter to vary in the simulation: Rb or dimension
set = 1 # 0: Rb, 1: dimension

# If set to vary Rb
if set == 0
    # Create an array of dimensions to iterate over
    dim_input = np.linspace(4, 18, 15)
    for i in 1:15
        # Call the plot function with each dimension value
        plot(Rb, pyconvert(Int64, dim_input[i]), Rb_arr, dimension_arr, HP_arr)
    end
# If set to vary dimension
elseif set == 1
    # Create an array of Rb values to iterate over
    Rb_input = np.linspace(1.2 * a, 7 * a, 14)
    println(length(Rb_input))
    for i in 1:14
        # Call the plot function with each Rb value
        plot(pyconvert(Float64, Rb_input[i-1]), dimension, Rb_arr, dimension_arr, HP_arr)
    end
end

# Example call to plot function (currently commented out)
# plot(2, 4, Rb_arr, dimension_arr, HP_arr)


In [ ]:
println(dimension_arr)
println(Rb_arr)
println(HP_arr)

plt.clf()
plt.plot(Rb_arr, HP_arr)
plt.title("5x5")
# plt.legend("")
plt.xlabel("Radius")
plt.ylabel("Hardness")
plt.savefig("/Users/kartikpingle/Desktop/5x5_new")